Version 1.0.1, Python 2.7.14, conda 4.3.34

In [1]:
import xgboost as xgb
import pandas as pd
import numpy as np
#import lightgbm as lgb
#from catboost import CatBoostRegressor
#from multiprocessing import *

#import sys
#from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

import matplotlib.pyplot as plt
%matplotlib inline

for p in [np, pd, xgb]:
    print (p.__name__, p.__version__)

('numpy', '1.14.2')
('pandas', u'0.22.0')
('xgboost', '0.7.post3')


Load the data:

In [2]:
from prepare_data import read_data
sales, items, categories, shops, test = read_data()

Create a data frame with all the informations given:

In [3]:
from prepare_data import create_df
df = create_df(sales, items, categories, shops)

Use the informations given in the sales:

In [4]:
from prepare_data import make_grid, get_aggregated, join_to_existing

index_cols = ['shop_id', 'item_id', 'date_block_num']

grid = make_grid(sales, index_cols)
gb = get_aggregated(sales, index_cols)
all_data = join_to_existing(grid, gb, index_cols)

In [5]:
all_data.sort_values(['date_block_num','shop_id','item_id'],inplace=True)
all_data = all_data.astype('int32')

In [6]:
all_data = all_data.assign(month = (all_data.date_block_num%12)+1).astype('int32')

all_data = all_data.assign(year = 2013)
all_data['year'] = np.where(all_data.date_block_num < 12, 2013, 2014)
all_data['year'] = np.where(all_data.date_block_num > 23, 2015, all_data.year).astype('int32')

all_data.head()

,shop_id,item_id,date_block_num,target,month,year
139255,0,19,0,0,1,2013
141495,0,27,0,0,1,2013
144968,0,28,0,0,1,2013
142661,0,29,0,0,1,2013
138947,0,32,0,6,1,2013


In [7]:
target_mean = all_data[['shop_id','item_id','target']].groupby(['shop_id','item_id'], as_index=False)[['target']].mean()
target_mean = target_mean.rename(columns={'target':'target_mean'}).astype('float32')

index_cols = ['shop_id', 'item_id']
all_data = join_to_existing(all_data, target_mean, index_cols)

In [8]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10913850 entries, 0 to 10913849
Data columns (total 7 columns):
shop_id           int64
item_id           int64
date_block_num    int32
target            int32
month             int32
year              int32
target_mean       float32
dtypes: float32(1), int32(4), int64(2)
memory usage: 458.0 MB


In [9]:
from prepare_data import k_fold_mean, loo_mean, smoothing_mean, expanding_mean

all_data, target_kf  = k_fold_mean(all_data)
all_data, target_loo = loo_mean(all_data)
all_data, target_sm = smoothing_mean(all_data)
all_data, target_exp = expanding_mean(all_data)
all_data.head()

,shop_id,item_id,date_block_num,target,month,year,target_mean,target_kfold,target_loo,target_smoothing,target_exp_mean
0,0.0,19.0,0.0,0.0,1.0,2013.0,0.0,0.334300,0.022727,0.237448,0.3343
1,0.0,27.0,0.0,0.0,1.0,2013.0,0.0,0.048523,0.056911,0.089905,0.3343
2,0.0,28.0,0.0,0.0,1.0,2013.0,0.0,0.142424,0.141414,0.168964,0.3343
3,0.0,29.0,0.0,0.0,1.0,2013.0,0.0,0.030303,0.037500,0.107910,0.3343
4,0.0,32.0,0.0,6.0,1.0,2013.0,8.0,0.894020,1.316088,1.260635,0.3343


In [ ]:
y_train = all_data.target.astype('int32')
X_train = all_data.drop(columns=['target'])
print(y_train.head())
print(X_train.head())

The test data must have the same features:

In [ ]:
from prepare_data import prepare_test
X_test = prepare_test(test)

X_test = join_to_existing(X_test, target_mean, index_cols)
X_test = join_to_existing(X_test, target_kf, index_cols)
X_test = join_to_existing(X_test, target_loo, index_cols)
X_test = join_to_existing(X_test, target_sm, index_cols)
X_test = join_to_existing(X_test, target_exp, index_cols)
X_test.head()

In [ ]:
#idx_item1 = df_train.item_price[df.item_price <= 0]
#idx_item2 = df_train.item_price[df.item_price > 100000]

#df_train.drop(df_train.index[idx_item2.index[0]], inplace=True)
#df_train.item_price.replace(to_replace=-1, value=2499.0 , inplace=True) 

In [ ]:
#df_train.drop(columns=['item_name', 'shop_name', 'date', 'item_category_name', 'day'], inplace=True)

In [ ]:
#print(df_train.item_price.min())
#print(df_train.item_price.max())

In [ ]:
#df_train.sort_values(by=['date_block_num'], axis=0, inplace=True)
#df_train.head()

In [ ]:
#df_train=df_train.groupby(['date_block_num', 'shop_id', 'item_id'], as_index=False).item_cnt_day.sum()
#df_monthly=df_monthly.rename(columns={'item_cnt_day': 'item_cnt_month'}).astype('int32')
#df_monthly=df_monthly.reset_index().astype('int32')

In [ ]:
#monthly_mean=df_monthly[['shop_id','item_id','item_cnt_month']].groupby(['shop_id','item_id'], as_index=False)[['item_cnt_month']].mean()
#monthly_mean=monthly_mean.rename(columns={'item_cnt_month':'mean_item_cnt'}).astype('float32')
#df_monthly=pd.merge(df_monthly, monthly_mean, how='left', on=['shop_id', 'item_id'])

In [ ]:
#prev_month = df_monthly
#df_monthly = df_monthly

In [ ]:
#monthly_sales=sales.groupby(["date_block_num","shop_id","item_id"])[
#    "date","item_price","item_cnt_day"].agg({"date":["min",'max'],"item_price":"mean","item_cnt_day":"sum"})

So, ready to train and validate, i.e. separate train data into train and validation set and train with xgboost or sth. else. Then prepare test data.

In [ ]:
#XGBoost
def xgb_rmse(preds, y):
    y = y.get_label()
    score = np.sqrt(metrics.mean_squared_error(y.clip(0.,20.), preds.clip(0.,20.)))
    return 'RMSE', score

params = {'eta': 0.2, 'max_depth': 4, 'objective': 'reg:linear', 'eval_metric': 'rmse', 'seed': 18, 'silent': True}

watchlist = [(xgb.DMatrix(x1, y1), 'train'), (xgb.DMatrix(x2, y2), 'valid')]
xgb_model = xgb.train(params, xgb.DMatrix(x1, y1), 100,  watchlist, verbose_eval=10, feval=xgb_rmse, maximize=False, early_stopping_rounds=20)
test['item_cnt_month'] = xgb_model.predict(xgb.DMatrix(test[col]), ntree_limit=xgb_model.best_ntree_limit)
test[['ID','item_cnt_month']].to_csv('xgb_submission.csv', index=False)

